In [1]:
# Test data prediction

# to mount Google Drive
from google.colab import drive
drive.mount('/gdrive')

import tensorflow as tf
tf.__version__

from google.colab import drive
drive.mount('/content/drive')

Mounted at /gdrive
Mounted at /content/drive


In [2]:
# libraries

import pandas as pd
import numpy as np
import os
import keras
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Dense,GlobalAveragePooling2D
from tensorflow.keras.applications import InceptionResNetV2, VGG16, ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam, SGD
from keras.models import load_model

In [13]:
# Model loading...

classes = ['0', '1', '2', '3', '4', '5']

model_to_use='resnet50'

base_model=ResNet50(weights='imagenet',include_top=False)
for layer in base_model.layers:
    layer.trainable=False

x=base_model.output
x=GlobalAveragePooling2D()(x)
x=Dense(128,activation='relu')(x)
x=Dense(128,activation='relu')(x)
preds=Dense(len(classes),activation='softmax')(x) 

model=Model(inputs=base_model.input, outputs=preds)

# pre-trained model (!)
model_path = '/content/drive/MyDrive/ICBA/Models/model-epoch-002-valacc-0.976103.h5'
model.load_weights(model_path)

In [15]:
# Image generator for test data

test_save = '/content/drive/MyDrive/ICBA/DataBase/ICBA/f4DL_2020_11_v3/Dataset4DL_v3/test'
print(test_save)

test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

test_generator=test_datagen.flow_from_directory(test_save, 
                                                target_size=(224,224),
                                                color_mode='rgb',
                                                batch_size=16,
                                                class_mode=None,
                                                shuffle=False)


/content/drive/MyDrive/ICBA/DataBase/ICBA/f4DL_2020_11_v3/Dataset4DL_v3/test
Found 1092 images belonging to 6 classes.


In [33]:
# data in order (?)

non_sorted = test_generator.filenames
sorted_list = sorted(non_sorted)

sorted_list == non_sorted

True

In [19]:
# Class per each file from test

ytrue_test = np.zeros(len(test_generator.filenames))

c1 = 0
for filenm in test_generator.filenames:
  file1 = filenm
  split = file1.split('/')]
  ytrue_test[c1] = split[0]
  c1 = c1 + 1

print(ytrue_test)
np.bincount(ytrue_test)

[0. 0. 0. ... 5. 5. 5.]


numpy.ndarray

In [22]:
from keras.utils import to_categorical

ypred_test = model.predict_generator(test_generator, steps=len(test_generator))
ypred_labels_test = np.argmax(ypred_test, axis=1)
print(ypred_labels_test)
ypred_onehot_test = to_categorical(ypred_labels_test, num_classes=6)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1905: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


[0 0 0 ... 5 5 5]


In [25]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

confusion_matrix(ytrue_test, ypred_labels_test)

array([[200,   0,   0,   0,   0,   0],
       [  5,  56,   0,   0,   0,   0],
       [  1,   0, 198,   0,   0,   0],
       [  0,   0,   0, 196,   0,   1],
       [  1,   0,   0,   5, 222,   5],
       [  0,   0,   0,   0,   0, 202]])

In [26]:
print(classification_report(ytrue_test, ypred_labels_test))

              precision    recall  f1-score   support

         0.0       0.97      1.00      0.98       200
         1.0       1.00      0.92      0.96        61
         2.0       1.00      0.99      1.00       199
         3.0       0.98      0.99      0.98       197
         4.0       1.00      0.95      0.98       233
         5.0       0.97      1.00      0.99       202

    accuracy                           0.98      1092
   macro avg       0.99      0.98      0.98      1092
weighted avg       0.98      0.98      0.98      1092



In [34]:
# Results.csv

import numpy as np
import pandas as pd

ytrue_test = np.zeros(len(test_generator.filenames), dtype=np.int)

c1 = 0
true_and_predict = np.zeros((len(test_generator.filenames),3), dtype=object)

filenames = test_generator.filenames

for filenm in test_generator.filenames:
  file1 = filenm
  split = file1.split('/')
  
  true = split[0]
  predict = np.argmax(ypred_test[c1])
  
  true_and_predict[c1,0] = filenm
  true_and_predict[c1,1] = true
  true_and_predict[c1,2] = predict
  c1 = c1 + 1

df = pd.DataFrame(true_and_predict,  columns=['class / filename','ytrue_test','ypredict_test'])
df.to_csv("results.csv")
print(df)


                                       class / filename  ... ypredict_test
0     0/uuid_0292c0d8-c25d-4a41-af86-37d46b2acebf-08...  ...             0
1     0/uuid_08dc04aa-bdbc-4492-b1b8-baa0cc801553-07...  ...             0
2     0/uuid_08dc04aa-bdbc-4492-b1b8-baa0cc801553-07...  ...             0
3     0/uuid_09ca0195-5847-446b-bac9-9eaa792aec00-07...  ...             0
4     0/uuid_0c18d787-3c11-4a2d-965f-6c7550e6b526-08...  ...             0
...                                                 ...  ...           ...
1087                                     5/IMG_2322.jpg  ...             5
1088                                     5/IMG_2327.jpg  ...             5
1089                                     5/IMG_2328.jpg  ...             5
1090                                     5/IMG_2329.jpg  ...             5
1091                                     5/IMG_2330.jpg  ...             5

[1092 rows x 3 columns]
